Using only numpy, design a simple neural network to classify the Iris flowers into 3 species based on sepal length, sepal width, petal length and petal width.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('Iris_Data.csv')

In [3]:
data['species'] = data['species'].astype('category').cat.codes

In [4]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X.mean(axis=0)) / X.std(axis=0)

y = np.eye(3)[y]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
input_size = X.shape[1]
hidden_size = 10
output_size = 3

In [7]:
np.random.seed(42)
W1 = np.random.randn(input_size, hidden_size)
b1 = np.zeros((1, hidden_size))
W2 = np.random.randn(hidden_size, output_size)
b2 = np.zeros((1, output_size))

In [8]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    return z * (1 - z)

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

In [9]:
def forward_propagation(X):
    Z1 = np.dot(X, W1) + b1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

In [10]:
def compute_loss(y_true, y_pred):
    return -np.mean(np.sum(y_true * np.log(y_pred + 1e-8), axis=1))

In [11]:
def backward_propagation(X, y_true, Z1, A1, Z2, A2):
    m = X.shape[0]

    dZ2 = A2 - y_true
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * sigmoid_derivative(A1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    return dW1, db1, dW2, db2

In [12]:
def update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    return W1, b1, W2, b2

In [13]:
num_epochs = 1000
learning_rate = 0.01

for epoch in range(num_epochs):
    Z1, A1, Z2, A2 = forward_propagation(X_train)

    loss = compute_loss(y_train, A2)

    dW1, db1, dW2, db2 = backward_propagation(X_train, y_train, Z1, A1, Z2, A2)

    W1, b1, W2, b2 = update_parameters(W1, b1, W2, b2, dW1, db1, dW2, db2, learning_rate)

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss:.4f}')

Epoch 0, Loss: 1.2222
Epoch 100, Loss: 0.7924
Epoch 200, Loss: 0.6423
Epoch 300, Loss: 0.5704
Epoch 400, Loss: 0.5239
Epoch 500, Loss: 0.4901
Epoch 600, Loss: 0.4639
Epoch 700, Loss: 0.4428
Epoch 800, Loss: 0.4253
Epoch 900, Loss: 0.4104


In [14]:
def predict(X):
    _, _, _, A2 = forward_propagation(X)
    return np.argmax(A2, axis=1)

test_predictions = predict(X_test)
test_accuracy = np.mean(test_predictions == np.argmax(y_test, axis=1))
print(f'Testing accuracy: {test_accuracy:.4f}')

Testing accuracy: 0.9333


In [15]:
def predict_species(sepal_length, sepal_width, petal_length, petal_width):
    input_data = np.array([sepal_length, sepal_width, petal_length, petal_width])
    input_data = (input_data - X_mean) / X_std
    input_data = input_data.reshape(1, -1)
    
    prediction = predict(input_data)
    species_mapping = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}
    return species_mapping[prediction[0]]

sepal_length = 5.1
sepal_width = 3.5
petal_length = 1.4
petal_width = 0.2

predicted_species = predict_species(sepal_length, sepal_width, petal_length, petal_width)
print(f'Predicted species: {predicted_species}')

Predicted species: setosa
